In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from datetime import datetime
from itertools import product

In [2]:
from ib_insync import *
util.startLoop()

ib=IB()

ib.connect('127.0.0.1', 7499, clientId=1) # kavi1234 TWS paper-trade
# ib.connect('127.0.0.1', 4004, clientId=2) # kavi1234 IBG paper-trade
# ib.connect('127.0.0.1', 4003, clientId=3) # ***kavi1234 IBG  LIVE ***
# ib.connect('127.0.0.1', 7498, clientId=4) # ***kavi1234 TWS  LIVE ***

<IB connected to 127.0.0.1:7499 clientId=1>

In [3]:
# Error handling
def onError(reqId, errorCode, errorString, contract):
    print("ERROR", reqId, errorCode, errorString)

# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

In [4]:
%%time
# **** Make list of qualified options stocks from the web (25 secs) *****

options_url1 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=1"
options_url2 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=2"
nse = pd.concat([pd.read_html(options_url1, header=0)[2], 
                         pd.read_html(options_url2, header=0)[2]], 
                        sort=True).drop_duplicates(subset='Symbol').reset_index(drop=True)

nse = nse.iloc[:, [1, 2, 3]]
nse.columns = ['iSymbol', 'Desc', 'Symbol']

# Note multipliers are not correct for NSE!. So get them from 5paisa!!!

paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol').reset_index(drop=True)  # It's the second table in the url
paisa_small = paisa[['Symbol', 'Mlot']].sort_values('Symbol') # small df only with Mlot

# Check for symbols missing in nse
# pd.merge(paisa_small, nse, on='Symbol', how='left', indicator=True).query("_merge == 'left_only'" )

nse_options = pd.merge(paisa_small, nse, on="Symbol", how='left')
nse_options["Exchange"] = 'NSE'

nse_options.loc[nse_options.iSymbol.isnull(), "iSymbol"] = nse_options['Symbol'].str.slice(0,9)

# List of nse indexes (extracted from error!)
idx_list = ["BANKNIFTY", "NIFTY50", "NSEFTSE", "USDINR", "INDIAVIX", 
            "NIFTYCPSE", "NIFTYINFR", "NIFTYIT", "NIFTYMID5", "NIFTYPSE"]
nse_options = nse_options.loc[~nse_options.iSymbol.isin(idx_list), :]

# Get the Stocks qualified
stocks = [Stock(symbol=x, exchange='NSE') for x in nse_options.iSymbol]
qual_stock = ib.qualifyContracts(*stocks)

# Put the contracts to a Contract column
nse_options = nse_options.assign(Contract = pd.Series(qual_stock).values)

Wall time: 19.6 s


In [5]:
# **** Get the close price of Underlying (30 milliseconds) ****
def get_price(contract):
    '''Gets the price of contract
    Args:
       (contract)    = object: qualified contract object
    
    Returns:
        price        = float
    '''
    
    m_data = ib.reqMktData(contract)
    
    while ib.waitOnUpdate():
        if m_data.marketPrice() == m_data.marketPrice():
            break
            
#     while m_data.close != m_data.close:
#         ib.sleep(0.01) #Wait until data is in.

    return m_data.close

In [14]:
list_of_prices = [get_price(c) for c in qual_stock[:5]]

ConnectionError: Not connected

In [10]:
list_of_prices

[1570.75, 200.35, 378.0, 32.4, 1155.3]

In [25]:
ib.disconnect()
while not ib.disconnectedEvent():
        break

In [26]:
ib.connect('127.0.0.1', 7499, clientId=1) # kavi1234 TWS paper-trade  

<IB connected to 127.0.0.1:7499 clientId=1>

In [16]:
help(ib.disconnectedEvent())

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).



In [ ]:
# Run it again!!!

list_of_prices = [get_price(c) for c in qual_stock]

# Update the dataframe
nse_options["Und_Price"] = list_of_prices

In [ ]:
%%time
# **** Get Expirations and Strikes. (4 mins)
contracts = [(i.symbol, '', i.secType, i.conId) for i in qual_stock]
contracts_p = [ib.reqSecDefOptParams(*p) for p in contracts]

# Filter symbol, expiration and strike from chains and make a cartesian
chains = [(i.tradingClass, i.expirations, i.strikes) for elem in contracts_p for i in elem]
L = [[[x[0]], sorted(x[1]), sorted(x[2])] for x in chains]

stock_chains = pd.DataFrame([j for i in L for j in product(*i)], 
                            columns=['Symbol', "Expiry", "Strike"])

df = pd.merge(stock_chains, nse_options, how='left', on='Symbol')

In [ ]:
%%time
# **** Get the top 2 out-of-SD-band contracts for each symbol, expiry and right ****

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

# Populate dte for standard deviation
df['DTE'] = [get_dte(d) for d in df.Expiry]
df['DTEMax'] = df.DTE.max()
df['Exchange'] = 'NSE'

# Make iSymbol unique for contracts
sd_df = df.drop_duplicates(subset=['iSymbol', 'DTE']).reset_index(drop=True)

# function to get greatest standard deviation between dte and dte_max
def get_stdev(contract, dte, dte_max):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
       (dte_max)  = int: max days to expiry
    Returns:
       maximum standard deviation in days (int)
    '''
    sd_days = str(dte_max)+' D'
    max_bars = ib.reqHistoricalData(contract=contract, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    
    sd_dte_max = np.std([b.close for b in max_bars], ddof=1)    # standard deviation of max dte
    sd_dte = np.std([b.close for b in max_bars[:-dte]], ddof=1)  # standard deviation of dte
    
    return max(sd_dte_max, sd_dte)  # returns the greatest standard deviation

sd_df['SD'] = pd.Series([get_stdev(c, d, x) for c, d, x in zip(sd_df.Contract, sd_df.DTE, sd_df.DTEMax)])

# Merge Standard Deviation to dataframe
df1 = pd.merge(df, sd_df, on=['iSymbol', 'Expiry'], how='left', suffixes=('', '_'))

df1 = df1.loc[:, ~df1.columns.str.endswith('_')].drop(['DTEMax'], axis=1)

# Designate Puts and Calls
df1['Right'] = np.where(df1.Strike < df1.Und_Price, 'P', 'C')

# Determine the Band Price
df1['Band_Price'] = np.where(df1.Right == 'P', df1.Und_Price - (df1.SD * put_sd), df1.Und_Price + (df1.SD * call_sd))

# Weed out-of-Band_Price strikes
mask = ((df1.Right == 'P') & (df1.Strike < df1.Band_Price)) | \
((df1.Right == 'C') & (df1.Strike > df1.Band_Price))

df2 = df1.loc[mask, :].reset_index(drop=True)  # Out-of-Band_Price

# Keep two out-of-band contracts for each right, expiry and symbol

df2["Band-Strike"] = pd.Series(abs(df2.Band_Price-df2.Strike)) # Calculate the absolute difference

df2 = df2.sort_values(by=['Symbol', 'Expiry', 'Right', 'Band-Strike']) # Sort
df3 = df2.groupby(['Symbol', 'Expiry', 'Right']).head(2).reset_index(drop=True) # Choose the top 2

In [ ]:
df4 = df3[['iSymbol', 'Expiry', 'Strike', 'Right', 'Exchange', 'Mlot', 'DTE']]

In [ ]:
df4 = df4.iloc[150:200, :] # !Data limiter

In [ ]:
option_contracts = [Option(symbol=s, exchange=x, lastTradeDateOrContractMonth=d, strike=t, right=r) for s, x, d, t, r in 
 zip(df4.iSymbol, df4.Exchange, df4.Expiry, df4.Strike, df4.Right)]

option_contract = [ib.qualifyContracts(i) for i in option_contracts]

# Put the contracts to a Option_Contract column
df4 = df4.assign(OptContract = pd.Series(option_contract).values)

# Remove the empty [] option contracts
df4 = df4.loc[df4.astype(str).OptContract != '[]', :]

In [ ]:
df4 = df4.head() # !Data Limiter

In [ ]:
# Margin check
o = [Order(action='SELL', orderType='MKT', totalQuantity=q) for q in df4.Mlot]
c = [optcontract for optcontract in df4.OptContract ]

df4 = df4.assign(Margin = [pd.to_numeric(ib.whatIfOrder(*c, o).maintMarginChange) for c, o in zip(c, o)])

In [ ]:
df4

In [ ]:
df4.OptContract

In [ ]:
ib.reqMktData(df4.OptContract[150][0], snapshot=True)

In [ ]:
[get_price(*c) for c in c]

In [ ]:
help(ib.sleep)

In [ ]:


# In the DataFrame
df4['Options'] = pd.Series(c)
df4['Orders'] = pd.Series(o)

# Margins
df4['Margin'] = [pd.to_numeric(ib.whatIfOrder(*c, o).maintMarginChange) for c, o in zip(c,o)]

# Option Price
df4['OptPrice'] = pd.Series([get_price(*i) for i in df4.Options])

# Return on Margin
df4['RoM'] = df4.OptPrice*df4.Mlot/df4.Margin*365/df4.DTE

In [ ]:
df3 = df3.sort_values(by=['RoM'], ascending=False)
calls_df = df3.loc[df3.Right == 'C', :]
puts_df = df3.loc[df3.Right == 'P', :]

x  ==  datetimedatetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_NSERaw.xlsx"
writer = pd.ExcelWriter(x)
df3.to_excel(writer, 'options', index=False, freeze_panes=(1, 1))
calls_df.to_excel(writer, 'calls', index=False, freeze_panes=(1,1))
puts_df.to_excel(writer, 'puts', index=False, freeze_panes=(1,1))
writer.save()

In [ ]:
# Function for logging errors
logging.basicConfig(filename='nse_options.log', level=logging.INFO)
def onError(reqId, errorCode, errorString):
    logging.info("ERROR", reqId, errorCode, errorString)

ib.setCallback('error', onError)   #get the function on the error

In [ ]:
ib.disconnect()

In [ ]:
ib.connect('127.0.0.1', 4004, clientId=2) # kavi1234 IBG paper-trade

In [ ]:
a